### 1. dark-patterns.csv 에서 string, type, predicate만 뽑아서 저장

- 중복 제거 시 완전히 동일한 문장만 제거
    - 공백/대소문자 차이까지는 묶지 않음

- 저장 csv 이름 : predicate_GT.csv

In [ ]:
import pandas as pd
import re

CSV_PATH = "/Users/soyoung/404DNF_AI/data/raw/dark-patterns.csv"  # 필요시 경로 수정

# 1) 읽기 + 필요한 컬럼만
df = pd.read_csv(CSV_PATH)[['Pattern String', 'Pattern Category', 'Pattern Type']].copy()
df['String'] = df['Pattern String'].astype(str).str.strip()

# 2) Pattern Category 정규화 후 우리가 원하는 4종만 남기기
WANTED = {"Urgency", "Misdirection", "Social Proof", "Scarcity"}
ALIASES = {
    "urgency": "Urgency",
    "misdirection": "Misdirection",
    "scarcity": "Scarcity",
    "socialproof": "Social Proof",
    "social-proof": "Social Proof",
    "social proof": "Social Proof",
}

def normalize_category(x):
    if pd.isna(x): 
        return None
    s = str(x).strip().lower()
    s = re.sub(r"\s+", " ", s)  # 여러 공백 정리
    s = s.replace("-", " - ").replace("  ", " ").replace(" - ", "-")  # 하이픈 주변 공백 정리
    if s in ALIASES:
        out = ALIASES[s]
    else:
        out = " ".join(w.capitalize() for w in s.split())
    return out if out in WANTED else None

df['_TypeNorm'] = df['Pattern Category'].map(normalize_category)
df = df[df['_TypeNorm'].notna()].copy()

# 3) 컬럼명/값 맞추기: Type은 정규화된 값으로, Predicate은 원래 'Pattern Type'
predicate_gt = (
    df.assign(Type=df['_TypeNorm'])
      .rename(columns={'Pattern Type': 'Predicate'})
      [['String', 'Type', 'Predicate']]
      .reset_index(drop=True)
)

# 중복 제거 전
# print(predicate_gt.head())
print(predicate_gt['Type'].value_counts())

# Type + String 기준으로 완전 동일한 중복 제거
predicate_gt_dedup = (
    predicate_gt
    .drop_duplicates(subset=['Type', 'String'])
    .reset_index(drop=True)
)

# (선택) 얼마나 줄었는지 확인
removed_by_type = (
    predicate_gt.groupby('Type')['String'].size()
    - predicate_gt_dedup.groupby('Type')['String'].size()
)

# 줄어든 개수 확인
print("Removed per Type:\n", removed_by_type.fillna(0).astype(int))
# 중복 제거 후 각 type별 개수 확인
print(predicate_gt_dedup['Type'].value_counts())


Type
Scarcity        679
Urgency         481
Social Proof    325
Misdirection    270
Name: count, dtype: int64
Removed per Type:
 Type
Misdirection     74
Scarcity        259
Social Proof     12
Urgency         270
Name: String, dtype: int64
Type
Scarcity        420
Social Proof    313
Urgency         211
Misdirection    196
Name: count, dtype: int64


In [17]:
from pathlib import Path

# 입력 CSV가 있는 폴더(data/raw)로 저장 경로 지정
out_dir = Path(CSV_PATH).parent
out_path = out_dir / "predicate_GT.csv"

# 정렬(옵션) 후 저장
predicate_gt_dedup.sort_values(['Type', 'String']).to_csv(
    out_path, index=False, encoding="utf-8-sig"
)

print(f"Saved: {out_path}")
print(predicate_gt_dedup.shape)
print(predicate_gt_dedup['Type'].value_counts())


Saved: /Users/soyoung/404DNF_AI/data/raw/predicate_GT.csv
(1140, 3)
Type
Scarcity        420
Social Proof    313
Urgency         211
Misdirection    196
Name: count, dtype: int64


### 2. datset.tsv에서 중복제거 하고 binary label만 두기

- 중복 제거 시 완전히 동일한 문장만 제거
    - 대소문자, 공백은 고려하지 않음

- 저장 csv 이름 : binary_label.csv

In [19]:
import pandas as pd
from pathlib import Path

# ===== 1) 입력 경로 =====
SRC_PATH = Path("/Users/soyoung/404DNF_AI/data/raw/dataset.tsv")
sep = "\t" if SRC_PATH.suffix.lower() in {".tsv", ".tab"} else ","

# ===== 2) 읽기 =====
df = pd.read_csv(SRC_PATH, sep=sep)

# ===== 3) (text, label) 컬럼 사용 =====
if not {"text", "label"}.issubset(df.columns):
    raise KeyError(f"'text', 'label' 컬럼이 필요합니다. 현재 컬럼: {list(df.columns)}")

binary_df = df[["text", "label"]].copy()

# ===== 4) 중복 제거 전 라벨 분포 =====
before_counts = binary_df["label"].value_counts().sort_index()

# ===== 5) 문장(text) 기준 완전 동일 중복 제거 =====
dup_mask = binary_df.duplicated(subset=["text"], keep="first")
removed_df = binary_df[dup_mask]                 # 제거될 행들
after_df   = binary_df[~dup_mask].reset_index(drop=True)

# ===== 6) 중복 제거 후 라벨 분포 =====
after_counts = after_df["label"].value_counts().sort_index()
removed_counts = removed_df["label"].value_counts().sort_index()

# ===== 7) 요약 출력 =====
labels = sorted(set(before_counts.index) | set(after_counts.index) | set(removed_counts.index))
summary = pd.DataFrame({
    "before": before_counts.reindex(labels, fill_value=0),
    "after":  after_counts.reindex(labels, fill_value=0),
    "removed": removed_counts.reindex(labels, fill_value=0),
}).astype(int)

print("Label counts (before / after / removed):")
print(summary)
print("\nTotals:",
      f"before={len(binary_df)}",
      f"after={len(after_df)}",
      f"removed={len(removed_df)}")

# ===== 8) 저장: 원본과 같은 폴더에 binary_label.csv =====
out_path = SRC_PATH.parent / "binary_label.csv"
after_df.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"\nSaved: {out_path}")


Label counts (before / after / removed):
       before  after  removed
label                        
0        1178   1178        0
1        1178   1178        0

Totals: before=2356 after=2356 removed=0

Saved: /Users/soyoung/404DNF_AI/data/raw/binary_label.csv


### 3. dataset.tsv, dark-patterns.csv merge

- Not Dark Pattern, Urgency, Misdirection, Scarcity, Social Proof 5가지 type만 저장
    - 중복 제거 시 완전히 동일한 문장만 제거, 띄어쓰기/대소문자에 대해서는 신경 쓰지 않음

- 저장 csv 이름 : merged.csv

In [20]:
import pandas as pd
import re
from pathlib import Path

# ===== 0) 경로 =====
SRC_TSV = Path("/Users/soyoung/404DNF_AI/data/raw/dataset.tsv")        # .tsv 원본
SRC_GT  = Path("/Users/soyoung/404DNF_AI/data/raw/predicate_GT.csv")   # predicate_GT.csv
sep_tsv = "\t" if SRC_TSV.suffix.lower() in {".tsv", ".tab"} else ","

# ===== 1) 공통: Type 정규화(대소문자/공백/하이픈) + 5개만 허용 =====
WANTED = {"Not Dark Pattern", "Urgency", "Misdirection", "Scarcity", "Social Proof"}
ALIASES = {
    # Not Dark Pattern
    "notdarkpattern": "Not Dark Pattern",
    "not-dark-pattern": "Not Dark Pattern",
    "not dark pattern": "Not Dark Pattern",
    "no dark pattern": "Not Dark Pattern",
    "non dark pattern": "Not Dark Pattern",
    # Others
    "urgency": "Urgency",
    "misdirection": "Misdirection",
    "scarcity": "Scarcity",
    "socialproof": "Social Proof",
    "social-proof": "Social Proof",
    "social proof": "Social Proof",
}

def normalize_type(x):
    if pd.isna(x):
        return None
    s = str(x).strip().lower()
    s = re.sub(r"\s+", " ", s)  # 여러 공백 → 한 칸
    # 하이픈 주변 불규칙 공백 정리
    s = s.replace("-", " - ").replace("  ", " ").replace(" - ", "-")
    if s in ALIASES:
        out = ALIASES[s]
    else:
        out = " ".join(w.capitalize() for w in s.split())
    return out if out in WANTED else None

# ===== 2) .tsv 정제: page_id 제거, text→String, Pattern Category→Type, 5개만 사용 =====
df_tsv = pd.read_csv(SRC_TSV, sep=sep_tsv)

# String 만들기 (text 우선, 없으면 예비 컬럼 시도)
if "text" in df_tsv.columns:
    string_series = df_tsv["text"].astype(str).str.strip()
elif "Pattern String" in df_tsv.columns:
    string_series = df_tsv["Pattern String"].astype(str).str.strip()
elif "String" in df_tsv.columns:
    string_series = df_tsv["String"].astype(str).str.strip()
else:
    raise KeyError(f"문장 컬럼을 찾지 못했습니다. 현재 컬럼: {list(df_tsv.columns)}")

# Type 만들기 (Pattern Category 우선, 없으면 label로 대체)
if "Pattern Category" in df_tsv.columns:
    type_series = df_tsv["Pattern Category"].map(normalize_type)
elif "label" in df_tsv.columns:
    type_series = df_tsv["label"].map(normalize_type)
else:
    raise KeyError("Type 정보를 만들 컬럼이 없습니다. ('Pattern Category' 또는 'label' 필요)")

df_tsv_out = pd.DataFrame({"String": string_series, "Type": type_series})
df_tsv_out = df_tsv_out[df_tsv_out["Type"].notna()].reset_index(drop=True)  # 5개만 남기기

# (참고) page_id는 최종 컬럼에 포함하지 않으므로 자연스럽게 제거됨

# ===== 3) predicate_GT.csv 정제: String, Type만 사용 + Type 재정규화로 5개만 유지 =====
df_gt = pd.read_csv(SRC_GT)

# 컬럼 이름 보정(혹시 'Pattern String', 'Pattern Category'로 되어 있다면)
if not {"String", "Type"}.issubset(df_gt.columns):
    df_gt = df_gt.rename(columns={
        "Pattern String": "String",
        "Pattern Category": "Type",
    })

if not {"String", "Type"}.issubset(df_gt.columns):
    raise KeyError(f"predicate_GT.csv에서 'String','Type' 컬럼을 찾지 못했습니다. 현재 컬럼: {list(df_gt.columns)}")

df_gt_out = df_gt[["String", "Type"]].copy()
df_gt_out["String"] = df_gt_out["String"].astype(str).str.strip()
df_gt_out["Type"]   = df_gt_out["Type"].map(normalize_type)
df_gt_out = df_gt_out[df_gt_out["Type"].notna()].reset_index(drop=True)   # 5개만 유지

# ===== 4) 머지(세로 결합) + 저장 =====
merged = pd.concat([df_tsv_out, df_gt_out], ignore_index=True)

out_path = SRC_TSV.parent / "merged.csv"   # .tsv와 같은 폴더에 저장
merged.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"Saved: {out_path}")
print("Shape:", merged.shape)
print("Type distribution:\n", merged["Type"].value_counts())


Saved: /Users/soyoung/404DNF_AI/data/raw/merged.csv
Shape: (3453, 2)
Type distribution:
 Type
Not Dark Pattern    1178
Scarcity             838
Social Proof         625
Urgency              421
Misdirection         391
Name: count, dtype: int64
